In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.streaming import *
from pyspark.sql.types import *

spark = SparkSession.builder.master("local").appName("Explode").getOrCreate()
sc = spark.sparkContext

In [8]:
# sql adaptive query execution adaptive.coalescePartitions.enabled will makr paritions dynamic
sc.setLogLevel("Error")
spark.conf.set("spark.sql.shuffle.partitions",3)
spark.conf.get("spark.sql.shuffle.partitions")
spark.conf.set("spark.sql.adaptive.enabled","false")
spark.conf.set("spark.sql.adaptive.coalescePartitions.enabled","false")

"""
Take care of null value as well - explode_outer
if we dont need positional index, we can go with explode_outer

posexplode_outer can be used to add positional index and cannot be used with withCoulmn
"""
filepath = "file:///C:/Users/venka/PycharmProjects/pythonProject/dataset/"
df = spark.read.option("delimiter","|").option("header",True).csv(filepath + "IntExplode.csv")
df.show()

+-----+---+-------------+
| Name|Age|    Education|
+-----+---+-------------+
|  Raj| 25|   MBA,BE,HSC|
|Shyam| 32|         NULL|
|Madhu| 47|ME,BE,Diploma|
+-----+---+-------------+



In [15]:
df.withColumn("Qualification",explode_outer(split("Education",","))).show()

+-----+---+-------------+-------------+
| Name|Age|    Education|Qualification|
+-----+---+-------------+-------------+
|  Raj| 25|   MBA,BE,HSC|          MBA|
|  Raj| 25|   MBA,BE,HSC|           BE|
|  Raj| 25|   MBA,BE,HSC|          HSC|
|Shyam| 32|         NULL|         NULL|
|Madhu| 47|ME,BE,Diploma|           ME|
|Madhu| 47|ME,BE,Diploma|           BE|
|Madhu| 47|ME,BE,Diploma|      Diploma|
+-----+---+-------------+-------------+



In [16]:
df.select("*",posexplode_outer(split("Education",","))) \
.withColumnRenamed("col","Qualification") \
.withColumnRenamed("pos","Index") \
.drop("Education").show()

+-----+---+-----+-------------+
| Name|Age|Index|Qualification|
+-----+---+-----+-------------+
|  Raj| 25|    0|          MBA|
|  Raj| 25|    1|           BE|
|  Raj| 25|    2|          HSC|
|Shyam| 32| NULL|         NULL|
|Madhu| 47|    0|           ME|
|Madhu| 47|    1|           BE|
|Madhu| 47|    2|      Diploma|
+-----+---+-----+-------------+

